In [1]:
from google.colab import files
df = files.upload()

Saving land_prices.csv to land_prices.csv


In [44]:
import pandas as pd
from io import BytesIO


In [45]:
df =pd.read_csv('/content/land_prices.csv')
df.head()

,Bodenrichtwert-Nummer,district_name,standard_land_value,typical_land_use_type,typical_floor_space_ratio,reference_date
0,1840,Friedrichshain-Kreuzberg,400.0,W - Wohngebiet,2.5,2012-01-01 00:00:00
1,1252,Reinickendorf,550.0,M2 - Mischgebiet,2,2012-01-01 00:00:00
2,1071,Friedrichshain-Kreuzberg,650.0,M2 - Mischgebiet,3,2012-01-01 00:00:00
3,1195,Tempelhof-Schöneberg,1300.0,M2 - Mischgebiet,3,2012-01-01 00:00:00
4,2279,Spandau,180.0,G - Gewerbe,-,2012-01-01 00:00:00


In [46]:
#removing unwanted columns
df =df.drop(columns=['Bodenrichtwert-Nummer'])



In [47]:
# Apply lowercasing to the column names and reassign to the DataFrame's columns
df.columns = df.columns.str.lower()

#changing the reference date column to remove time part
df['reference_date'] = pd.to_datetime(df['reference_date']).dt.date

In [48]:
df.head()

,district_name,standard_land_value,typical_land_use_type,typical_floor_space_ratio,reference_date
0,Friedrichshain-Kreuzberg,400.0,W - Wohngebiet,2.5,2012-01-01
1,Reinickendorf,550.0,M2 - Mischgebiet,2,2012-01-01
2,Friedrichshain-Kreuzberg,650.0,M2 - Mischgebiet,3,2012-01-01
3,Tempelhof-Schöneberg,1300.0,M2 - Mischgebiet,3,2012-01-01
4,Spandau,180.0,G - Gewerbe,-,2012-01-01


In [49]:
#renaming 'district_name' to 'neighborhood_name'
df.rename(columns={'district_name': 'neighborhood_name'}, inplace=True)
df.head()

,neighborhood_name,standard_land_value,typical_land_use_type,typical_floor_space_ratio,reference_date
0,Friedrichshain-Kreuzberg,400.0,W - Wohngebiet,2.5,2012-01-01
1,Reinickendorf,550.0,M2 - Mischgebiet,2,2012-01-01
2,Friedrichshain-Kreuzberg,650.0,M2 - Mischgebiet,3,2012-01-01
3,Tempelhof-Schöneberg,1300.0,M2 - Mischgebiet,3,2012-01-01
4,Spandau,180.0,G - Gewerbe,-,2012-01-01


In [50]:
!pip install -q pandasql


In [51]:
import pandas as pd
from pandasql import sqldf


In [52]:
query = """
SELECT DISTINCT typical_land_use_type
FROM df
"""
result = sqldf(query)
print(result)

                                typical_land_use_type
0                                      W - Wohngebiet
1                                    M2 - Mischgebiet
2                                         G - Gewerbe
3                                     M1 - Kerngebiet
4                          M1* - Einzelhandelszentrum
5                                   GB - Gemeinbedarf
6                           Gp - Gewerbe produzierend
7                                    F - Forstflächen
8                           LW-EGA - Erwerbsgartenbau
9                                     LW-G - Grünland
10                                       LW-A - Acker
11                                   LW-GR - Grünland
12                              LW-GR - Grünland (GR)
13                SF-FH - Sonstige Flächen - Friedhof
14  LF-F - Flächen der Land-o. Forstwirtschaft - f...
15           S-WO - Sonderbaufläche - Wochenendhäuser
16                          G - Gewerbliche Baufläche
17   S-FZT - Sonderbaufläche

In [53]:
query = """
SELECT
  CASE WHEN typical_land_use_type IN ('W - Wohngebiet' ,'S-WO - Sonderbaufläche - Wochenendhäuser', 'W-EFH - Wohnbaufläche - Ein- und Zweifamilienhäuser', 'W - Wohnbaufläche', ' W-MFH - Wohnbaufläche - Mehrfamilienhäuser') THEN 'Residential Area'
       WHEN typical_land_use_type IN ('G - Gewerbe', 'M1* - Einzelhandelszentrum', 'Gp - Gewerbe produzierend', 'G-GD - Gewerbliche Baufläche - Handel and dienstleistungsorientiertes Gewerbe', ' GE - Gewerbegebiet', 'G-MES - Gewerbliche Baufläche - Messen, Ausstellungen u.a.', 'M-BGH - Gemischte Baufläche - Büro und Geschäftshäuser' ) THEN 'Commercial Area'
       WHEN typical_land_use_type In ('F - Forstflächen', 'LW-EGA - Erwerbsgartenbau', 'LW-G - Grünland', 'LW-A - Acker', 'LW-GR - Grünland', 'LW-GR - Grünland (GR)', 'LF-F - Flächen der Land-o. Forstwirtschaft - forstwirtschaftliche Fläche', 'LF-GR - Flächen der Land-o. Forstwirtschaft - Grünland', 'LF-A - Flächen der Land-o. Forstwirtschaft - Acker', 'LF-EGA - Flächen der Land-o. Forstwirtschaft - Erwerbsgartenbaufläche') Then 'Forest and Agricultural Area'
       WHEN typical_land_use_type In ('M2 - Mischgebiet', 'GB - Gemeinbedarf', 'M-BGH - Gemischte Baufläche - Büro und Geschäftshäuser') Then 'Mixed Area'
       ELSE 'Others'
       END AS land_use_category
FROM df
"""
result = sqldf(query)
print(result)

      land_use_category
0      Residential Area
1            Mixed Area
2            Mixed Area
3            Mixed Area
4       Commercial Area
...                 ...
16821        Mixed Area
16822        Mixed Area
16823        Mixed Area
16824        Mixed Area
16825        Mixed Area

[16826 rows x 1 columns]


In [37]:
df['land_use_category'] = result
df.head()


,neighborhood_name,standard_land_value,typical_land_use_type,typical_floor_space_ratio,reference_date,land_use_category
0,Friedrichshain-Kreuzberg,400.0,W - Wohngebiet,2.5,2012-01-01,Residential Area
1,Reinickendorf,550.0,M2 - Mischgebiet,2,2012-01-01,Mixed Area
2,Friedrichshain-Kreuzberg,650.0,M2 - Mischgebiet,3,2012-01-01,Mixed Area
3,Tempelhof-Schöneberg,1300.0,M2 - Mischgebiet,3,2012-01-01,Mixed Area
4,Spandau,180.0,G - Gewerbe,-,2012-01-01,Commercial Area


In [54]:
#checking for null values
df.isnull().values.any()


np.False_

In [55]:
#replacing 'keine daten' with nan
mask = df.isin(['keine daten'])
columns_with_keine_daten = mask.any()  # True for columns that contain it
columns_with_keine_daten = columns_with_keine_daten[columns_with_keine_daten].index.tolist()
print("Columns containing 'keine daten':", columns_with_keine_daten)

Columns containing 'keine daten': []


In [56]:
import numpy as np
df.replace('keine daten', np.nan, inplace=True)


In [57]:
df.head()

,neighborhood_name,standard_land_value,typical_land_use_type,typical_floor_space_ratio,reference_date
0,Friedrichshain-Kreuzberg,400.0,W - Wohngebiet,2.5,2012-01-01
1,Reinickendorf,550.0,M2 - Mischgebiet,2,2012-01-01
2,Friedrichshain-Kreuzberg,650.0,M2 - Mischgebiet,3,2012-01-01
3,Tempelhof-Schöneberg,1300.0,M2 - Mischgebiet,3,2012-01-01
4,Spandau,180.0,G - Gewerbe,-,2012-01-01


In [43]:
#cleaned land prices csv downlaod
df.to_csv('cleaned_land_prices.csv', index=False)
from google.colab import files
files.download('cleaned_land_prices.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>